<a href="https://colab.research.google.com/github/biswajitmohanty/genai/blob/main/Simple_RNN_Predict_7_days_Stock_Price_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
import plotly.graph_objects as go

def fetch_stock_data(ticker, period="5y"):
    stock = yf.Ticker(ticker)
    df = stock.history(period=period)
    return df[["Close"]]

def create_sequences(data, time_steps=60, future_steps=1):
    X, y = [], []
    for i in range(len(data) - time_steps - future_steps + 1):
        X.append(data[i:i+time_steps])
        y.append(data[i+time_steps:i+time_steps+future_steps])
    return np.array(X), np.array(y)

def build_rnn_model(time_steps, future_steps):
    model = Sequential([
        SimpleRNN(50, activation='relu', return_sequences=True, input_shape=(time_steps, 1)),
        Dropout(0.2),
        SimpleRNN(50, activation='relu'),
        Dropout(0.2),
        Dense(future_steps)
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

def train_and_predict(ticker, time_steps=60, future_steps=1, epochs=10):
    df = fetch_stock_data(ticker)
    scaler = MinMaxScaler()
    df_scaled = scaler.fit_transform(df)

    X, y = create_sequences(df_scaled, time_steps, future_steps)

    model = build_rnn_model(time_steps, future_steps)
    model.fit(X, y, epochs=epochs, batch_size=32, verbose=1)

    last_sequence = df_scaled[-time_steps:].reshape(1, time_steps, 1)
    prediction = model.predict(last_sequence)
    predicted_prices = scaler.inverse_transform(prediction)

    return predicted_prices

# Example usage
time_steps_options = [30, 60, 90]
ticker = "TSLA"
predictions = {}

for time_steps in time_steps_options:
    predictions[time_steps] = train_and_predict(ticker, time_steps, future_steps=7)

# Save to CSV
for time_steps, pred in predictions.items():
    pd.DataFrame(pred, columns=[f"Day {i+1}" for i in range(7)]).to_csv(f"{ticker}_predictions_{time_steps}.csv", index=False)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - loss: 0.0990
Epoch 2/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0225
Epoch 3/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.0164
Epoch 4/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.0137
Epoch 5/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.0104
Epoch 6/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0109
Epoch 7/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0091
Epoch 8/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0082
Epoch 9/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0075
Epoch 10/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0081
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 157ms/step
Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


38/38 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - loss: 0.1762
Epoch 2/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0324
Epoch 3/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0212
Epoch 4/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0178
Epoch 5/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0134
Epoch 6/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0123
Epoch 7/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.0106
Epoch 8/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 0.0104
Epoch 9/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - loss: 0.0090
Epoch 10/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - loss: 0.0089
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 257ms/step
Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


37/37 ━━━━━━━━━━━━━━━━━━━━ 4s 40ms/step - loss: 0.1772
Epoch 2/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - loss: 0.0261
Epoch 3/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step - loss: 0.0187
Epoch 4/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - loss: 0.0156
Epoch 5/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 2s 34ms/step - loss: 0.0121
Epoch 6/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - loss: 0.0103
Epoch 7/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - loss: 0.0097
Epoch 8/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - loss: 0.0085
Epoch 9/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - loss: 0.0078
Epoch 10/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 0.0079
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 165ms/step
